# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import time
# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity_map= city_data_df.hvplot.points('Lat','Lng', geo=True, tiles = 'OSM', frame_width = 800, frame_height=800, size='Huminity')

# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lat,Lng]

In [4]:
### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
cities_narrow_df = city_data_df[(city_data_df['Max Temp']>20)]

# Drop any rows with null values
cities_ideal = cities_narrow_df.dropna()

# Display sample data
cities_ideal

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
7,7,carnarvon,-24.8667,113.6333,20.02,76,0,7.46,AU,1666108232
9,9,saint-pierre,-21.3393,55.4781,22.82,68,0,5.14,RE,1666108236
10,10,colorado,-22.8375,-51.9731,20.36,86,96,4.69,BR,1666107965
12,12,trairi,-3.2778,-39.2689,31.07,51,34,8.09,BR,1666108038
...,...,...,...,...,...,...,...,...,...,...
567,567,sangamner,19.5667,74.2167,23.82,85,100,2.28,IN,1666108559
568,568,road town,18.4167,-64.6167,31.02,70,40,4.12,VG,1666108560
569,569,ranong,9.9658,98.6348,24.34,93,100,1.44,TH,1666108330
574,574,canchungo,12.0672,-16.0333,35.07,61,67,2.91,GW,1666108565


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=cities_ideal[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,
7,carnarvon,AU,-24.8667,113.6333,76,
9,saint-pierre,RE,-21.3393,55.4781,68,
10,colorado,BR,-22.8375,-51.9731,86,
12,trairi,BR,-3.2778,-39.2689,51,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000
limit = 20
params = {
    "api_key":geoapify_key,
    "limit": limit,
    "categories":  "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{latitude},{longitude},{radius}'
    params["bias"] = f'proximity:{latitude},{longitude}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary

    # Convert the API response to JSON format

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        response = requests.get(base_url, params=params)
        name_address = response.json()

        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        if "features" in name_address and name_address["features"]:
            hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
faya - nearest hotel: Traktir 333
carnarvon - nearest hotel: No hotel found
saint-pierre - nearest hotel: No hotel found
colorado - nearest hotel: No hotel found
trairi - nearest hotel: No hotel found
puerto ayora - nearest hotel: No hotel found
avarua - nearest hotel: No hotel found
guiratinga - nearest hotel: No hotel found
rikitea - nearest hotel: No hotel found
bambous virieux - nearest hotel: No hotel found
saint-louis - nearest hotel: No hotel found
cape town - nearest hotel: No hotel found
naberera - nearest hotel: Las Mayorales
labuhan - nearest hotel: No hotel found
tomatlan - nearest hotel: No hotel found
sao joao da barra - nearest hotel: No hotel found
kapaa - nearest hotel: No hotel found
mogadishu - nearest hotel: Hôtel Colorado
hilo - nearest hotel: No hotel found
businga - nearest hotel: No hotel found
lasa - nearest hotel: No hotel found
vaini - nearest hotel: No hotel found
cayenne - nearest hotel: No hotel found
east london - nearest hotel: No h

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,Traktir 333
7,carnarvon,AU,-24.8667,113.6333,76,No hotel found
9,saint-pierre,RE,-21.3393,55.4781,68,No hotel found
10,colorado,BR,-22.8375,-51.9731,86,No hotel found
12,trairi,BR,-3.2778,-39.2689,51,No hotel found
...,...,...,...,...,...,...
567,sangamner,IN,19.5667,74.2167,85,No hotel found
568,road town,VG,18.4167,-64.6167,70,No hotel found
569,ranong,TH,9.9658,98.6348,93,No hotel found
574,canchungo,GW,12.0672,-16.0333,61,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display

# Configure the map plot
hotel_map= hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size ="lat",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel name", "Country"]
)
    

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country)